### 引用Nuget包

In [1]:
#r "nuget:Senparc.Weixin.WxOpen"
#r "nuget:Senparc.Weixin.Cache.Redis"
#r "nuget:Senparc.Weixin.MP"
#r "nuget:Microsoft.Extensions.Caching.Memory,6.0.0"
#r "nuget:Microsoft.Extensions.Configuration.Json,6.0.0"

Installed Packages Microsoft.Extensions.Caching.Memory, 6.0.0 Microsoft.Extensions.Configuration.Json, 6.0.0 Senparc.Weixin.Cache.Redis, 2.15.13 Senparc.Weixin.MP, 16.19.3 Senparc.Weixin.WxOpen, 3.16.3

### 引用命名空间

In [2]:
using Senparc.CO2NET;
using Senparc.CO2NET.RegisterServices;
using Senparc.Weixin.Cache.Redis;
using Senparc.Weixin.Entities;
using Senparc.Weixin;
using Senparc.Weixin.MP;
using Senparc.Weixin.WxOpen;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Configuration;
using Senparc.Weixin.WxOpen.AdvancedAPIs.Template;
using Senparc.Weixin.MP.AdvancedAPIs.TemplateMessage;

### 配置文件

In [3]:
var configBuilder = new ConfigurationBuilder();
// 此处需要使用配置文件的绝对路径
configBuilder.AddJsonFile(@"D:\PythonProjects\senparc.json", false, false);

### 注册盛派SDK

In [4]:
// 关闭盛派APM
Senparc.CO2NET.APM.Config.EnableAPM = false;
// 不抛出异常
Senparc.Weixin.Config.ThrownWhenJsonResultFaild = false;

var config = configBuilder.Build();

var senparcSetting = new SenparcSetting();
var senparcWeixinSetting = new SenparcWeixinSetting();

config.GetSection("SenparcSetting").Bind(senparcSetting);
config.GetSection("SenparcWeixinSetting").Bind(senparcWeixinSetting);

var services = new ServiceCollection();
services.AddSenparcGlobalServices(config);//Senparc.CO2NET 全局注册

IRegisterService register = RegisterService.Start(senparcSetting)
                                            .UseSenparcGlobal();

var redisConfigurationStr = senparcSetting.Cache_Redis_Configuration;
Senparc.CO2NET.Cache.Redis.Register.SetConfigurationOption(redisConfigurationStr);
//键值对缓存策略（推荐）
Senparc.CO2NET.Cache.Redis.Register.UseKeyValueRedisNow();

register.UseSenparcWeixinCacheRedis();

register.UseSenparcWeixin(senparcWeixinSetting)
        .RegisterMpAccount(senparcWeixinSetting, "梧州珍宝巴士")
        .RegisterWxOpenAccount(senparcWeixinSetting, "梧州公交");

### 发送统一服务消息

In [5]:
public static readonly string WxOpenAppId = Senparc.Weixin.Config.SenparcWeixinSetting.WxOpenAppId;// 与微信小程序后台的AppId设置保持一致，区分大小写。
public static readonly string WxOpenAppSecret = Senparc.Weixin.Config.SenparcWeixinSetting.WxOpenAppSecret;// 与微信小程序账号后台的AppId设置保持一致，区分大小写。
public static readonly string WeixinAppId = Senparc.Weixin.Config.SenparcWeixinSetting.WeixinAppId;// 微信公众号AppId
public static readonly string WeixinAppSecret = Senparc.Weixin.Config.SenparcWeixinSetting.WeixinAppSecret;// 微信公众号AppId
private const string TemplateId = "OJ-cY-IJhkw8SF_l0OZK_Cy5cKGN-uqhMiSXLxCaON8";// 模板Id
const string openid = "oMr9M5SoFzyGqjHm2pvUImj1w0BQ";// 接收测试消息的openid

UniformSendData sendData = new(openid,
                    new Mp_Template_Msg(WeixinAppId, TemplateId, "https://mp.weixin.qq.com/s/ObnyNhpOIjij7zgS1RIvaA",
                    new Miniprogram_Page(WxOpenAppId, string.Empty),
                    new
                    {
                        keyword1 = new TemplateDataItem("3075309865"),
                        keyword2 = new TemplateDataItem("2023-09-10 17:02:22"),
                        keyword3 = new TemplateDataItem("49路"),
                        keyword4 = new TemplateDataItem("010678"),
                        keyword5 = new TemplateDataItem("2.00")
                    }));

var result = TemplateApi.UniformSend(WxOpenAppId, sendData);
// 查看返回结果
result.Display();

errcode,45109
ErrorCodeValue,45109
errmsg,"This API has been unsupported. For more details, please view https://developers.weixin.qq.com/community/develop/doc/000ae8d6348af08e7030bc2546bc01?blockType=1 rid: 650b8665-65fe50fc-076f7520"
P2PData,<null>


### 发送模板消息

In [6]:
var result = Senparc.Weixin.MP.AdvancedAPIs.TemplateApi.SendTemplateMessage(WeixinAppId, openid, TemplateId,
                "https://mp.weixin.qq.com/s/ObnyNhpOIjij7zgS1RIvaA",
                new
                {
                    keyword1 = new TemplateDataItem("3075309865"),
                    keyword2 = new TemplateDataItem("2023-09-10 17:02:22"),
                    keyword3 = new TemplateDataItem("49路"),
                    keyword4 = new TemplateDataItem("010678"),
                    keyword5 = new TemplateDataItem("2.00")
                });
result.Display();

msgid,0
errcode,不合法的OpenID
ErrorCodeValue,40003
errmsg,invalid openid rid: 650b8666-127fe826-7727fb1e
P2PData,<null>
